# 1 - Web Scraping / Scrapeo con BeatifulSoup (Bs4)
<p align = 'center'>
<img src = 'https://editor.analyticsvidhya.com/uploads/75676cover.jfif'/>
</p>

Web scraping o raspado web, es una técnica utilizada mediante programas de software para extraer información de sitios web de manera automatizada y masiva. Usualmente, estos programas simulan la navegación de un humano en la web ya sea utilizando el protocolo HTTP manualmente, o incrustando un navegador en una aplicación.

El web scraping está muy relacionado con la indexación de la web, la cual indexa la información de la web utilizando un robot y es una técnica universal adoptada por la mayoría de los motores de búsqueda. Sin embargo, el web scraping se enfoca más en la transformación de datos sin estructura en la web, como el formato HTML, en datos estructurados que pueden ser almacenados y analizados en una base de datos central, en una hoja de cálculo o en alguna otra fuente de almacenamiento. Alguno de los usos del web scraping son la comparación de precios en tiendas, la monitorización de datos relacionados con el clima de cierta región, la detección de cambios en sitios webs y la integración de datos en sitios webs.

En los últimos años el web scraping se ha convertido en una **técnica muy utilizada** dentro del sector del posicionamiento web gracias a su capacidad de generar grandes cantidades de datos para crear contenidos de calidad.

Podríamos pensar que el web scraping es nuestro recurso a **falta de una API o un feed RSS**. A falta de una fuente de datos, siempre podemos extraer aquello que sale por pantalla.

Todo aquello que necesite interactividad con la página web va a necesitar **selenium**, una librería que nos permite darle instrucciones a nuestro explorador para poder simular los movimientos que haríamos por dentro de la web:
 - Cambiar de página
 - Rellenar formularios
 - Hacer click en botones
 - Cerrar pop-ups
 - Rechazar cookies
 
Por otro lado, todo aquello que sea estático podemos tirar únicamente de **BeautifulSoup**, la primera librería para scrapeo que vamos a conocer que nos permite, una vez cargada la página de la que vamos a extraer la información, recorrer el código HTML y quedarnos con aquellos datos que necesitemos.

Tendremos que realizar scrapeo cuando no dispongamos de una fuente más sencilla de la que obtener información como una API, una base de datos, fichero, etc...

# 2 - Extracción de los datos del HTML

Para scrapear necesitamos saber que pinta tiene la **estructura general** que tiene un HTML.

El HTML consiste en contenido `<etiquetado>`, es como si fueran cajas de contenido, organizado de manera jerárquica:

```
<html>
    <head>
        <title>Titulo de la pagina</title>
    </head>
    <body>
        <h1>Cabecera</h1>
        <p>Parrafo</p>
    </body>
</html>
```

$$$$

Las etiquetas el HTML se pueden clasificar en varios grupos, dependiendo del tipo de contenido que posea. Estos son algunos ejemplos:

+ cabecera: `<h1>`, `<h2>`, `<h3>`, `<hgroup>`...
+ texto: `<b>`, `<p>`, `<span>`...
+ embebido: `<audio>`, `<img>`, `<video>`...
+ tabular: `<table>`, `<tr>`, `<td>`, `<tbody>`...
+ secciones: `<header>`, `<section>`, `<article>`...
+ metadata: `<meta>`, `<title>`, `<script>`...

$$$$


Las etiquetas pueden tener atributos. Por ejemplo:

`<div class="text-monospace" id="name_132", href="www.example.com"> Contenido de la pagina </div>`

Esta etiqueta `div` tiene los siguientes atributos:

+ class: atributo con valor "text-monospace". La clase no es única en la página, varios elementos pueden tener la misma clase.
+ id: atributo con valor "name_132". El id de una etiqueta la identifica de manera unívoca, no puede haber dos etiquetas con el mismo id.
+ href: atributo con valor "www.example.com". El href suele contener el link a otra parte de la página.

Siguiendo con la analogía de las cajas, si una etiqueta de HTML es una caja, los atributos serían las pegatinas pegadas en la tapa de la caja.

Conociendo cual es el contenido que queremos extraer, debemos encontrar las **etiquetas** que nos interesan dentro de todo el HTML de la página web.

Para ello vamos a usar **[BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)**.

In [ ]:
#%pip install beautifulsoup4

In [1]:
import requests as req
from bs4 import BeautifulSoup as bs #este alias es standard

# 3 - Scrapeando **WIKIPEDIA**

Vamos a echar un ojo a esta web **[Países europeos según esperanza de vida](https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy)** para intentar sacar la información de la tabla que contiene la esperanza de vida de los países europeos.

Hasta ahora habíamos visto `requests`para hacer llamadas a url (normalmente endpoints de las APIS que hemos estado mirando)

¿Y si llamamos a una url de una web digamos `normal`?

In [ ]:
#url de la pagina a scrapear
url = 'https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy'

html = req.get(url).text #obtenemos el contenido de la pagina
html[:1000] #mostramos los primeros 1000 caracteres

In [ ]:
str(html).split('/head>')[1][:1000] #mostramos los primeros 1000 caracteres de la seccion body / Esto es una gualtrapada para ver el codigo html de la pagina

In [4]:
# cantidad de caracteres que tiene esta parte del codigo html
len(str(html).split('/head>')[1])

632766

Esto es infumable y para nada práctico. Para ello BeautifulSoup entra en juego para facilitarnos la vida.

Vimos en la docu, la manera de pasarle un `html` para que nos lo parsee (traduzca/convierta) en un objeto de BeautifulSoup muchos más manejable, amigable y con muchos métodos que nos facilitan el poder navegar el `html` para encontrar el contenido exacto que queremos extraer.

Así, al **parsearlo**, es mucho más **legible**. A este objeto por convención se le llama `soup`.

In [5]:
soup = bs(html, 'html.parser') #creamos el objeto soup, y le decimos que lo que le pasamos tiene pinta de html
type(soup)

bs4.BeautifulSoup

In [ ]:
print(soup.prettify())

Vamos a acceder a la tabla que hay en la web. 

En primer lugar es necesario entender cómo se estructura una tabla en código HTML.

Gracias a BeautifulSoup podemos encontrar los diferentes **elementos** de una manera muy **veloz**.

Hay varias funciones para encontrar elementos concretos. `find()` y `find_all()` son dos ejemplos.

Vamos a ver como es la **estructura general** de una tabla en html:

![](https://www.corelangs.com/html/tables/img/html-table-structure.png)

![](https://www.ourtutorials.in/html/img/table1.JPG)

In [ ]:
# Tabla
tabla = soup.find('table') #buscamos la tabla, en este caso solo hay una. Si hubiera mas, solo nos devolveria la primera. Si queremos todas, usamos find_all
# soup.find_all('table')  -->  En caso de que tuviéramos más de una tabla esta sería la línea de código correcta para acceder a todas ellas
tabla

In [9]:
# cuantas tablas hay...?
len(soup.find_all('table'))

15

In [ ]:
tabla.prettify()

In [ ]:
print(tabla.prettify()[:1000])

In [13]:
# Filas de la tabla
filas = tabla.find_all('tr') # tr --> table row
len(filas) #Acordaros que vimos que find all nos devolvía una lista con todos los elementos encontrados

59

In [14]:
type(filas) # ResultSet es una lista

bs4.element.ResultSet

Al ser una lista puedo recorrer cada elemento.

In [ ]:
# Texto del primer elemento de la lista
filas[0].text

In [11]:
print(filas[0].text)


Countries andterritories

Life expectancy for population in general

Life expectancy for male

Life expectancy for female

Sex gap

Population(thous.)





In [16]:
# texto de cada fila solo para los 3 primeros elementos
for f in filas[:3]:
    print(type(f.text), f.text)
    print('-----------------')

<class 'str'> 
Countries andterritories

Life expectancy for population in general

Life expectancy for male

Life expectancy for female

Sex gap

Population(thous.)



-----------------
<class 'str'> 
at birth

bonus0→15

at 15

bonus15→65

at 65

bonus65→80

at 80

at birth

at 15

at 65

at 80

at birth

at 15

at 65

at 80

at birth

at 15

at 65

at 80

-----------------
<class 'str'> 
 Monaco
86.37
0.45
71.82
1.40
23.21
3.09
11.30
84.45
69.90
21.56
10.14
88.50
73.95
25.05
12.50
4.06
4.05
3.49
2.36
39


-----------------


In [17]:
#vamos a sacar el texto de 3 primeros elementos del cuerpo de la tabla
for f in filas[2:5]:
    print(type(f.text), f.text)
    print('-----------------')

<class 'str'> 
 Monaco
86.37
0.45
71.82
1.40
23.21
3.09
11.30
84.45
69.90
21.56
10.14
88.50
73.95
25.05
12.50
4.06
4.05
3.49
2.36
39


-----------------
<class 'str'> 
 San Marino
85.71
0.15
70.86
1.78
22.64
2.88
10.52
84.21
69.36
21.25
9.52
87.10
72.24
23.90
11.31
2.89
2.88
2.65
1.79
34


-----------------
<class 'str'> 
 Andorra
84.04
0.58
69.62
1.85
21.47
3.72
10.19
82.10
67.68
19.81
9.03
86.11
71.70
23.18
11.16
4.01
4.02
3.36
2.13
81


-----------------


In [18]:
f.text

'\n\xa0Andorra\n84.04\n0.58\n69.62\n1.85\n21.47\n3.72\n10.19\n82.10\n67.68\n19.81\n9.03\n86.11\n71.70\n23.18\n11.16\n4.01\n4.02\n3.36\n2.13\n81\n\n'

In [14]:
f.text.split('\n')

['',
 '\xa0Andorra',
 '84.04',
 '0.58',
 '69.62',
 '1.85',
 '21.47',
 '3.72',
 '10.19',
 '82.10',
 '67.68',
 '19.81',
 '9.03',
 '86.11',
 '71.70',
 '23.18',
 '11.16',
 '4.01',
 '4.02',
 '3.36',
 '2.13',
 '81',
 '',
 '']

In [19]:
# Hagamos una lista, que contenga una lista por fila
# strip() --> quita los espacios que haya delante y detrás del texto
# split() --> va a separar la lista por la secuencia de caracteres '\n' para hacer una lista
filas_limpias = [f.text.strip().split('\n') for f in filas]
filas_limpias[:5]

[['Countries andterritories',
  '',
  'Life expectancy for population in general',
  '',
  'Life expectancy for male',
  '',
  'Life expectancy for female',
  '',
  'Sex gap',
  '',
  'Population(thous.)'],
 ['at birth',
  '',
  'bonus0→15',
  '',
  'at 15',
  '',
  'bonus15→65',
  '',
  'at 65',
  '',
  'bonus65→80',
  '',
  'at 80',
  '',
  'at birth',
  '',
  'at 15',
  '',
  'at 65',
  '',
  'at 80',
  '',
  'at birth',
  '',
  'at 15',
  '',
  'at 65',
  '',
  'at 80',
  '',
  'at birth',
  '',
  'at 15',
  '',
  'at 65',
  '',
  'at 80'],
 ['Monaco',
  '86.37',
  '0.45',
  '71.82',
  '1.40',
  '23.21',
  '3.09',
  '11.30',
  '84.45',
  '69.90',
  '21.56',
  '10.14',
  '88.50',
  '73.95',
  '25.05',
  '12.50',
  '4.06',
  '4.05',
  '3.49',
  '2.36',
  '39'],
 ['San Marino',
  '85.71',
  '0.15',
  '70.86',
  '1.78',
  '22.64',
  '2.88',
  '10.52',
  '84.21',
  '69.36',
  '21.25',
  '9.52',
  '87.10',
  '72.24',
  '23.90',
  '11.31',
  '2.89',
  '2.88',
  '2.65',
  '1.79',
  '34'],


In [20]:
# La primera fila es la cabecera, y hay que limpiar los espacios en blanco
cabecera = filas_limpias[0]
cabecera

['Countries andterritories',
 '',
 'Life expectancy for population in general',
 '',
 'Life expectancy for male',
 '',
 'Life expectancy for female',
 '',
 'Sex gap',
 '',
 'Population(thous.)']

In [21]:
# vamos a arreglarlo
final = [] # aquí el resultado final limpio
for fila in filas_limpias:
    tmp = [] # lista vacía temporal
    for elemento in fila:
        if elemento != '':
            tmp.append(elemento)
    final.append(tmp)

final[:5]

[['Countries andterritories',
  'Life expectancy for population in general',
  'Life expectancy for male',
  'Life expectancy for female',
  'Sex gap',
  'Population(thous.)'],
 ['at birth',
  'bonus0→15',
  'at 15',
  'bonus15→65',
  'at 65',
  'bonus65→80',
  'at 80',
  'at birth',
  'at 15',
  'at 65',
  'at 80',
  'at birth',
  'at 15',
  'at 65',
  'at 80',
  'at birth',
  'at 15',
  'at 65',
  'at 80'],
 ['Monaco',
  '86.37',
  '0.45',
  '71.82',
  '1.40',
  '23.21',
  '3.09',
  '11.30',
  '84.45',
  '69.90',
  '21.56',
  '10.14',
  '88.50',
  '73.95',
  '25.05',
  '12.50',
  '4.06',
  '4.05',
  '3.49',
  '2.36',
  '39'],
 ['San Marino',
  '85.71',
  '0.15',
  '70.86',
  '1.78',
  '22.64',
  '2.88',
  '10.52',
  '84.21',
  '69.36',
  '21.25',
  '9.52',
  '87.10',
  '72.24',
  '23.90',
  '11.31',
  '2.89',
  '2.88',
  '2.65',
  '1.79',
  '34'],
 ['Andorra',
  '84.04',
  '0.58',
  '69.62',
  '1.85',
  '21.47',
  '3.72',
  '10.19',
  '82.10',
  '67.68',
  '19.81',
  '9.03',
  '86.11

Hasta ahora hemos cogido la información contenida en una página que tiene una tabla. Hemos identificado la tabla y hemos accedido a ella. Con `find()` hemos buscado la etiqueta que nos interesa para ello (`<table>`). Una vez obtenido el HTML únicamente de la tabla lo hemos traducido para que sea algo más legible y hemos arreglado su contenido para poder llegar al siguiente paso: crear nuestro DataFrame.

In [22]:
final[0] # primer elemento de la lista final

['Countries andterritories',
 'Life expectancy for population in general',
 'Life expectancy for male',
 'Life expectancy for female',
 'Sex gap',
 'Population(thous.)']

In [23]:
final[1] # segundo elemento de la lista final

['at birth',
 'bonus0→15',
 'at 15',
 'bonus15→65',
 'at 65',
 'bonus65→80',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80']

In [24]:
final[2]

['Monaco',
 '86.37',
 '0.45',
 '71.82',
 '1.40',
 '23.21',
 '3.09',
 '11.30',
 '84.45',
 '69.90',
 '21.56',
 '10.14',
 '88.50',
 '73.95',
 '25.05',
 '12.50',
 '4.06',
 '4.05',
 '3.49',
 '2.36',
 '39']

Para poder crear el DataFrame a partir de listas es necesario que todas ellas tengan la **misma cantidad de elementos**, tanto la que dará nombre a las columnas como la que contendrá la información de las filas.

In [25]:
import pandas as pd

In [26]:
nombres_columnas = final[1] # la primera fila la omitimos, nos quedamos la segunda, pero tenemos que arreglarla
nombres_columnas

['at birth',
 'bonus0→15',
 'at 15',
 'bonus15→65',
 'at 65',
 'bonus65→80',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80']

In [27]:
data = final[2:] # desde la primera fila con información, la de Liechtenstein, hasta el final

In [28]:
len(data[0]), len(nombres_columnas) # las filas tienen info para más columnas que la lista que dará nombre a las columnas...

(21, 19)

En este caso no coinciden. Así que tendremos que hacer algún pequeño arreglo extra antes de conseguir hacer el DataFrame.

In [29]:
nombres_columnas

['at birth',
 'bonus0→15',
 'at 15',
 'bonus15→65',
 'at 65',
 'bonus65→80',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80']

In [30]:
data[0]

['Monaco',
 '86.37',
 '0.45',
 '71.82',
 '1.40',
 '23.21',
 '3.09',
 '11.30',
 '84.45',
 '69.90',
 '21.56',
 '10.14',
 '88.50',
 '73.95',
 '25.05',
 '12.50',
 '4.06',
 '4.05',
 '3.49',
 '2.36',
 '39']

In [31]:
nombres_columnas.insert(0, 'Country') # al añadir en primer lugar 'Country' ya tenemos tantos elementos como en el resto, que serán las filas

In [33]:
nombres_columnas.append('Poblacion')

In [34]:
df = pd.DataFrame(data, columns = nombres_columnas)

In [35]:
df.head(10)

,Country,at birth,bonus0→15,at 15,bonus15→65,at 65,bonus65→80,at 80,at birth,at 15,...,at 80,at birth,at 15,at 65,at 80,at birth,at 15,at 65,at 80,Poblacion
0,Monaco,86.37,0.45,71.82,1.40,23.21,3.09,11.30,84.45,69.90,...,10.14,88.50,73.95,25.05,12.50,4.06,4.05,3.49,2.36,39
1,San Marino,85.71,0.15,70.86,1.78,22.64,2.88,10.52,84.21,69.36,...,9.52,87.10,72.24,23.90,11.31,2.89,2.88,2.65,1.79,34
2,Andorra,84.04,0.58,69.62,1.85,21.47,3.72,10.19,82.10,67.68,...,9.03,86.11,71.70,23.18,11.16,4.01,4.02,3.36,2.13,81
3,Switzerland,83.95,0.36,69.31,2.20,21.51,3.46,9.97,82.01,67.38,...,9.08,85.83,71.17,22.83,10.63,3.82,3.80,2.78,1.54,8871
4,Italy,83.72,0.26,68.98,2.33,21.31,3.46,9.77,81.57,66.84,...,8.77,85.75,70.99,22.76,10.50,4.18,4.15,3.09,1.73,59499
5,Spain,83.67,0.29,68.96,2.58,21.54,3.57,10.11,80.96,66.26,...,8.96,86.31,71.58,23.40,10.94,5.35,5.32,3.92,1.98,47912
6,Liechtenstein,83.63,0.62,69.25,1.92,21.17,3.81,9.98,81.84,67.43,...,8.91,85.32,70.97,22.58,10.75,3.48,3.54,2.96,1.84,40
7,Gibraltar,83.55,1.05,69.60,2.66,22.26,3.70,10.96,80.92,67.08,...,9.61,86.14,72.05,23.95,11.93,5.23,4.97,3.56,2.31,38
8,France,83.33,0.39,68.72,3.31,22.03,3.58,10.61,80.43,65.85,...,9.41,86.09,71.44,23.78,11.45,5.66,5.59,3.78,2.04,66439
9,Norway,83.31,0.25,68.56,2.19,20.75,3.53,9.28,81.75,67.02,...,8.37,84.85,70.07,21.84,10.02,3.10,3.05,2.27,1.65,5519


In [ ]:
import requests as req


In [ ]:
#RETO

#Vamos a escrapear la tabla de esta web --> https://en.wikipedia.org/wiki/List_of_countries_by_GNI_(nominal)_per_capita
#Escrapeamos la primera tabla, y cuando lo tengamos vamos con la segunda
#La salida debe ser un dataframe limpito,

In [ ]:
html= req.get(https://en.wikipedia.org/wiki/List_of_countries_by_GNI_(nominal)_per_capita).tex

# 4 - Geolocalizando una IP

In [2]:
import requests as req
from bs4 import BeautifulSoup as bs

**¿Dónde estoy?** --> https://tools.keycdn.com/geo

Esta web nos da toda la información de la IP que introducimos.

In [3]:
url='https://tools.keycdn.com/geo'

In [4]:
html = req.get(url).content # hace lo mismo que .text

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

WTF? Que ha pasado



que es un [user agent] ??(https://www.zenrows.com/blog/user-agent-web-scraping#what-is)

User agents for web scraping --> [link](https://useragentstring.com/pages/Browserlist/)

In [8]:
headers = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/6.0 Mobile/10A5355d Safari/8536.25'}

response = req.get('http://httpbin.org/headers', headers=headers)

print(response.status_code)
print(response.text)

200
{
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Host": "httpbin.org", 
    "User-Agent": "Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/6.0 Mobile/10A5355d Safari/8536.25", 
    "X-Amzn-Trace-Id": "Root=1-66e878e5-639424233f3ff4780b79f889"
  }
}



Se ha tenido que añadir un diccionario que contiene 'User-agent'. Hay muchos sitios web que van a poner impedimentos para que podamos obtener la información a través del scrapeo. Estamos haciendo una petición sin cabecera y no puede identificar el navegador que se está utilizando para acceder.

Cuando accedemos a una web pedimos acceso aportando mucha información en los metadatos: quién soy, dónde estoy...

Al hacer un `get` a pelo no está la información del navegador, se nos detecta como bot y se nos corta el acceso.

Esta información se contiene en lo que se llama **cabecera**. Tal como indico en el diccionario le estoy diciendo a esta web que estoy accediendo desde el navegador Mozilla en su versión 5.0. Estoy intentando sacar información de la web que, a priori, no debería poder sacar. Esta web pone un impedimento para ello, pero hay una solución, la cabecera (el **`header=`**).


In [1]:
http://httpbin.org/headers

SyntaxError: invalid syntax (832054228.py, line 1)

In [ ]:
https://useragentstring.com/pages/Browserlist/

In [75]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

html = req.get('https://tools.keycdn.com/geo', headers=headers)

print(html.status_code)
print(html.text)


200

<!DOCTYPE html>
<html lang="en" prefix="og: https://ogp.me/ns#">
    <head>
        <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
        <meta name="version" content="57ed0a7e3b29ae8f3a50b7d383902359f80ff602">

        <title>IP Location Finder - IP Lookup With Detailed Geolocation Data | KeyCDN Tools</title>
        <meta name="description" content="What is my IP address? Lookup an IP address or hostname to get the geographical location.">

        <meta name="twitter:card" content="summary_large_image">
        <meta name="twitter:site" content="@KeyCDN">
        <meta name="twitter:creator" content="@KeyCDN">
        <meta property="og:url" content="https://tools.keycdn.com/geo">
        <meta property="og:type" content="website">
        <meta property="og:title" content="IP Location Finder - IP Lookup With Detailed Geolocation Data | KeyCDN Tools">
        <meta property="og:description" content="What is

In [78]:
# soup parseada para tener el HTML legible
soup = bs(html.text, 'html.parser')

In [79]:
print(soup.prettify()[:1000]) # con .prettify() lo que hago es poner el código todavía más legible, con indentación
                       # .prettify() funciona distinto dentro que fuera de la función print()

<!DOCTYPE html>
<html lang="en" prefix="og: https://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <meta content="57ed0a7e3b29ae8f3a50b7d383902359f80ff602" name="version"/>
  <title>
   IP Location Finder - IP Lookup With Detailed Geolocation Data | KeyCDN Tools
  </title>
  <meta content="What is my IP address? Lookup an IP address or hostname to get the geographical location." name="description"/>
  <meta content="summary_large_image" name="twitter:card"/>
  <meta content="@KeyCDN" name="twitter:site"/>
  <meta content="@KeyCDN" name="twitter:creator"/>
  <meta content="https://tools.keycdn.com/geo" property="og:url"/>
  <meta content="website" property="og:type"/>
  <meta content="IP Location Finder - IP Lookup With Detailed Geolocation Data | KeyCDN Tools" property="og:title"/>
  <meta content="What is my IP address? Lookup an IP address or hostname to get the geographical location." propert

Utilizando el inspector en el navegador podemos activar el cursor y encontrar en qué parte del código HTML se encuentra el elemento que necesitamos. Una vez identificado lo buscamos en nuestra *sopa*

In [80]:
print(soup.find('div', {'id': 'geoResult'}).prettify()) # conseguido con el inspector

<div class="mt-4" id="geoResult">
 <div class="bg-light medium rounded p-3">
  <p class="small text-uppercase text-muted font-weight-semi-bold line-height-headings mb-2">
   Location
  </p>
  <dl class="row mb-0">
   <dt class="col-4">
    City
   </dt>
   <dd class="col-8 text-monospace">
    Madrid
   </dd>
   <dt class="col-4">
    Region
   </dt>
   <dd class="col-8 text-monospace">
    Madrid (M)
   </dd>
   <dt class="col-4">
    Postal code
   </dt>
   <dd class="col-8 text-monospace">
    28026
   </dd>
   <dt class="col-4">
    Country
   </dt>
   <dd class="col-8 text-monospace">
    Spain (ES)
   </dd>
   <dt class="col-4">
    Continent
   </dt>
   <dd class="col-8 text-monospace">
    Europe (EU)
   </dd>
   <dt class="col-4">
    Coordinates
   </dt>
   <dd class="col-8 text-monospace">
    40.3907 (lat) / -3.6997 (long)
   </dd>
   <dt class="col-4">
    Time
   </dt>
   <dd class="col-8 text-monospace">
    2024-09-13 20:49:30 (Europe/Madrid)
   </dd>
  </dl>
  <p class

In [81]:
tabla = soup.find('div', {'id': 'geoResult'}) # le llamo tabla, pues acabará siendo nuestra tabla
info = tabla.find('div', {'class': 'bg-light medium rounded p-3'}) # la información que tendrá la tabla está aquí

Ahora que ya sé dónde está la información que necesito y tengo una variable que la alberga puedo buscar más profundamente hasta poder extraer los datos que necesito.

Toda la info la tengo dentro de ese `<div>` veo que lo tengo en una lista con la class='row mb-0'

Pero cuidado porque hay dos clases iguales.

Accedamos a info para que nos dé todos los dd que tenga la clase col-8 text-monospace. Así me va a dar todos los que hay con la misma etiqueta y la misma clase.

Por comodidad de lectura del código esto lo guardamos en una variable que se llama detalles, al fin y al cabo son los detalles de la información que contendrá la tabla que estamos haciendo.

In [82]:
detalles = info.find_all('dd', {'class': 'col-8 text-monospace'}) #Elemento y añado tb la clase

Hago una lista con el texto que hay en cada elemento de detalles. Le llamo conexión pues son los detalles de la info de esa conexión IP.

In [84]:
conexion = [e.text for e in detalles]
conexion

['Madrid',
 'Madrid (M)',
 '28026',
 'Spain (ES)',
 'Europe (EU)',
 '40.3907 (lat) / -3.6997 (long)',
 '2024-09-13 20:49:30 (Europe/Madrid)',
 '47.61.8.143',
 '47-61-8-143.red-acceso.airtel.net',
 'Vodafone Spain',
 '12430']

Necesito hacer un diccionario. `<dd>` es la información de la conexión. `<dt>` tiene el nombre de cada una de las filas (es la columna de la izquierda). Por ello cojo esa información y lo uso como `keys`, mientras que lo que hemos hecho hasta ahora serán los `values`.

In [85]:
keys = [e.text for e in tabla.find_all('dt')]
keys

['City',
 'Region',
 'Postal code',
 'Country',
 'Continent',
 'Coordinates',
 'Time',
 'IP address',
 'Hostname',
 'Provider',
 'ASN']

In [86]:
dicc = {k:v for k,v in zip(keys,conexion)} #me hago un diccionario con los datos de la conexion

In [87]:
dicc

{'City': 'Madrid',
 'Region': 'Madrid (M)',
 'Postal code': '28026',
 'Country': 'Spain (ES)',
 'Continent': 'Europe (EU)',
 'Coordinates': '40.3907 (lat) / -3.6997 (long)',
 'Time': '2024-09-13 20:49:30 (Europe/Madrid)',
 'IP address': '47.61.8.143',
 'Hostname': '47-61-8-143.red-acceso.airtel.net',
 'Provider': 'Vodafone Spain',
 'ASN': '12430'}

In [88]:
import pandas as pd
ip = pd.DataFrame(dicc, index = [0]) #al ser valores escalares hay que proporcionar índice
                                     #al aportar índice se genera una única fila
ip

,City,Region,Postal code,Country,Continent,Coordinates,Time,IP address,Hostname,Provider,ASN
0,Madrid,Madrid (M),28026,Spain (ES),Europe (EU),40.3907 (lat) / -3.6997 (long),2024-09-13 20:49:30 (Europe/Madrid),47.61.8.143,47-61-8-143.red-acceso.airtel.net,Vodafone Spain,12430


**Búsqueda según IP**

https://tools.keycdn.com/geo?host=137.255.90.7

In [90]:
#Mi url ha cambiado
url_ip = 'https://tools.keycdn.com/geo?host=88.30.60.7'

In [9]:
url_ip='https://tools.keycdn.com/geo?host=137.255.90.7' #tengo parámetros en la direccion en funcion de la IP que busque...🤔

html = req.get(url_ip, headers = headers).content

soup=bs(html, 'html.parser')

In [92]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en" prefix="og: https://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <meta content="57ed0a7e3b29ae8f3a50b7d383902359f80ff602" name="version"/>
  <title>
   IP Location Finder - IP Lookup With Detailed Geolocation Data | KeyCDN Tools
  </title>
  <meta content="What is my IP address? Lookup an IP address or hostname to get the geographical location." name="description"/>
  <meta content="summary_large_image" name="twitter:card"/>
  <meta content="@KeyCDN" name="twitter:site"/>
  <meta content="@KeyCDN" name="twitter:creator"/>
  <meta content="https://tools.keycdn.com/geo?host=137.255.90.7" property="og:url"/>
  <meta content="website" property="og:type"/>
  <meta content="IP Location Finder - IP Lookup With Detailed Geolocation Data | KeyCDN Tools" property="og:title"/>
  <meta content="What is my IP address? Lookup an IP address or hostname to get the geographical 

Es igual que antes, pues la estructura es la misma, primero saco todos los `<dd>`luego todos los `<dt>`

In [93]:
tabla = soup.find('div', {'id': 'geoResult'})
tabla.find_all('dd', {'class': 'col-8 text-monospace'})

[<dd class="col-8 text-monospace">Benin (BJ)</dd>,
 <dd class="col-8 text-monospace">Africa (AF)</dd>,
 <dd class="col-8 text-monospace">9.5 (lat) / 2.25 (long)</dd>,
 <dd class="col-8 text-monospace">2024-09-13 20:13:20 (Africa/Porto-Novo)</dd>,
 <dd class="col-8 text-monospace">137.255.90.7</dd>,
 <dd class="col-8 text-monospace">137.255.90.7</dd>,
 <dd class="col-8 text-monospace">SBIN</dd>,
 <dd class="col-8 text-monospace">28683</dd>]

In [94]:
tabla.find_all('dt')

[<dt class="col-4">Country</dt>,
 <dt class="col-4">Continent</dt>,
 <dt class="col-4">Coordinates</dt>,
 <dt class="col-4">Time</dt>,
 <dt class="col-4">IP address</dt>,
 <dt class="col-4">Hostname</dt>,
 <dt class="col-4">Provider</dt>,
 <dt class="col-4">ASN</dt>]

In [ ]:
#'{:2.2f}'.format(12.5436363636363)  # formato en strings de numeros

Puedo hacer una función que haga de golpe todo lo que hemos hecho hasta ahora con una dirección IP

In [10]:
def geo(ip):

    url = f'https://tools.keycdn.com/geo?host={ip}' #Dinamica en fucnion de la IP que quiero buscar

    html = req.get(url, headers = headers).content

    soup = bs(html, 'html.parser')

    tabla = soup.find('div', {'id': 'geoResult'})

    conexion = [e.text for e in tabla.find_all('dd', {'class': 'col-8 text-monospace'})]

    return conexion

Vimos que una url también puede tener diferentes parámetros. ¿Qué pasa si tengo una lista de IP de la que quiero obtener la información y hacerme un dataframe?

In [11]:
list_ip = ['137.255.90.7', '195.255.90.10', '177.255.21.7']

Sacad un DataFrame que tenga la información de las direcciones IP contenidas en `list_ip`:
1. probad a hacer vuestra propia funcion
2. haced el scrapeo "manual"
3. hacedlo utilizando la funcion

In [20]:
def geo_completa(ip):

    url = f'https://tools.keycdn.com/geo?host={ip}' #Dinamica en fucnion de la IP que quiero buscar

    html = req.get(url, headers = headers).content

    soup = bs(html, 'html.parser')

    tabla = soup.find('div', {'id': 'geoResult'})

    valores = [e.text for e in tabla.find_all('dd', {'class': 'col-8 text-monospace'})]
    columnas = [e.text for e in tabla.find_all('dt', {'class': 'col-4'})]

    dicc_respuesta = {k:v for k,v in zip(columnas, valores)}

    return dicc_respuesta

In [22]:
# Inténtalo tú (alumnos)
resultados = []

for ip in list_ip :
    resultados.append(geo_completa(ip))

list

### Resultado con una simple llamada a una función

In [ ]:
# @title
def geolocalizacion_ip(lista):
  #crea una lista vacía en la que va a ir metiendo cada diccionario
  lista_data= []

  #bucle que itera cada elemento de la lista que entra en la función
  for ip in lista:
    url = f'https://tools.keycdn.com/geo?host={ip}' #Dinamica en fucnion de la IP que quiero buscar

    html = req.get(url, headers = user_agent).content

    soup = bs(html, 'html.parser')

    tabla = soup.find('div', {'id': 'geoResult'})

    #hago una lista con la información de cada fila
    informacion = [e.text for e in tabla.find_all('dd', {'class': 'col-8 text-monospace'})]

    #hago una lista con la información de las columnas (que usaré como keys del diccionario)
    columnas = [e.text for e in tabla.find_all('dt', {'class': 'col-4'})]

    #hago un diccionario que una vez hecho meto en la lista vacía del principio
    data= dict(zip(columnas,informacion))
    lista_data.append(data)

  #fuera del bucle genero el DataFrame con la lista de diccionarios, dando como índices la lista que entra
  #así cada fila tendrá como índice su ip
  df= pd.DataFrame(lista_data, index= lista)


  return df

In [ ]:
geolocalizacion_ip(list_ip).T # con .T pivoto el DataFrame

### RECORDATORIO de como hacer un df desde una lista de diccionarios

In [27]:
import pandas as pd

dicc1 = {'a': 1, 'b': 2, 'c': 3}
dicc2 = {'a': 23, 'b': 52, 'd': 666}
lista_diccionarios = [dicc1, dicc2]

df = pd.DataFrame(lista_diccionarios)
df

,a,b,c,d
0,1,2,3.0,NaN
1,23,52,NaN,666.0


In [29]:
resultados_en_df =  pd.DataFrame(resultados)
resultados_en_df

,Country,Continent,Coordinates,Time,IP address,Hostname,Provider,ASN,City,Region,Postal code
0,Benin (BJ),Africa (AF),9.5 (lat) / 2.25 (long),2024-09-16 19:56:21 (Africa/Porto-Novo),137.255.90.7,137.255.90.7,SBIN,28683,NaN,NaN,NaN
1,Finland (FI),Europe (EU),60.1797 (lat) / 24.9344 (long),2024-09-16 21:56:22 (Europe/Helsinki),195.255.90.10,ws10.lonne.fi,Elisa Oyj,719,Helsinki,Uusimaa (18),00141
2,Colombia (CO),South America (SA),10.4013 (lat) / -75.5267 (long),2024-09-16 13:56:22 (America/Bogota),177.255.21.7,dinamic-tigo-177-255-21-7.tigo.com.co,Colombia Movil,27831,Cartagena,Departamento de Bolívar (BOL),130005


In [30]:
resultados

[{'Country': 'Benin (BJ)',
  'Continent': 'Africa (AF)',
  'Coordinates': '9.5 (lat) / 2.25 (long)',
  'Time': '2024-09-16 19:56:21 (Africa/Porto-Novo)',
  'IP address': '137.255.90.7',
  'Hostname': '137.255.90.7',
  'Provider': 'SBIN',
  'ASN': '28683'},
 {'City': 'Helsinki',
  'Region': 'Uusimaa (18)',
  'Postal code': '00141',
  'Country': 'Finland (FI)',
  'Continent': 'Europe (EU)',
  'Coordinates': '60.1797 (lat) / 24.9344 (long)',
  'Time': '2024-09-16 21:56:22 (Europe/Helsinki)',
  'IP address': '195.255.90.10',
  'Hostname': 'ws10.lonne.fi',
  'Provider': 'Elisa Oyj',
  'ASN': '719'},
 {'City': 'Cartagena',
  'Region': 'Departamento de Bolívar (BOL)',
  'Postal code': '130005',
  'Country': 'Colombia (CO)',
  'Continent': 'South America (SA)',
  'Coordinates': '10.4013 (lat) / -75.5267 (long)',
  'Time': '2024-09-16 13:56:22 (America/Bogota)',
  'IP address': '177.255.21.7',
  'Hostname': 'dinamic-tigo-177-255-21-7.tigo.com.co',
  'Provider': 'Colombia Movil',
  'ASN': '27831

### Ejemplo LinkedIn

In [1]:
#Vamos intentar scrapear Linkedin
URL='https://www.linkedin.com/jobs/search/' #Esta es la direccion de la pagina de busqueda de empleo de Linkedin general

A esta direccion le podemos poner parametros para afinar nuestra búsqueda.

URL = 'https://www.linkedin.com/jobs/search/?keywords=data&location=Madrid&refresh=true'

`?keywords=data` nos dice que la palabra clave que queremos buscar es 'data'
    
`&location=Madrid` nos dice que queremos buscar en Madrid
    
`&refresh=true` nos dice que queremos que nos devuelva los resultados más recientes

Podemos filtrar más nuestra búsqueda en la web y veremos que van apareciendo más ***parametros*** en la URL.

`&f_TPR=r120960` nos dice que empleos de la ultima semana. El parámetro va en segundos así que 60 * 60 * 24 * número de días atrás que queremos buscar

`&start={i*25}` donde i sería el número de página


In [2]:
#Vamos a intentar scrapear esta URL
import requests as req
from bs4 import BeautifulSoup as bs
url = 'https://www.linkedin.com/jobs/search/?keywords=data&location=Espa%C3%B1a&refresh=true%27&position=1&pageNum=0'
html = req.get(url).content
soup=bs(html, 'html.parser')

In [3]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="d_jobs_guest_search" name="pageKey"/>
  <!-- -->
  <meta content="urlType=jserp_custom;emptyResult=false" name="linkedin:pageTag"/>
  <meta content="en_US" name="locale"/>
  <meta data-app-version="2.0.1974" data-browser-id="ca69b50f-30fa-44b6-8d38-57625717d7fb" data-call-tree-id="AAYiVJtL6boBB275VJqSIA==" data-dfp-member-lix-treatment="control" data-disable-jsbeacon-pagekey-suffix="false" data-dna-member-lix-treatment="control" data-enable-page-view-heartbeat-tracking="" data-human-member-lix-treatment="control" data-member-id="0" data-multiproduct-name="jobs-guest-frontend" data-page-instance="urn:li:page:d_jobs_guest_search;8yQ52lj2TcqiaWxh8A0V9A==" data-service-name="jobs-guest-frontend" id="config"/>
  <link href="https://www.linkedin.com/jobs/data-jobs" rel="canonical"/>
  <!-- -->
  <!-- -->
  <!-- -->
  <!-- -->
  <!-- -->
  <!-- -->
  <link href="https://static.licdn.com/aero-v1/sc/h/al2o9zrvru7aqj8e1x2rzsrca" rel="icon

In [17]:
# Intentad sacar los ingredientes de la recete de esta sopa y obtener para cada oferta de trabajo:

lista_ofertas = []
#Listado de ofertas
ofertas = soup.find_all('div', {'class':'base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card'})

for oferta in ofertas[:3]:
    dicc_oferta = {} #Diccionario donde almacenamos datos de 1 oferta
    #Titulo oferta
    titulo_oferta = oferta.find('span', {'class':'sr-only'}).text.strip()
    dicc_oferta['titulo_oferta'] = titulo_oferta

    # - Empresa
    empresa = oferta.find('a', {'class':'hidden-nested-link'}).text.strip()
    dicc_oferta['empresa'] = empresa
    
    # - Ubicacion
    ubicacion = oferta.find('span', {'class':'job-search-card__location'}).text.strip()
    dicc_oferta['ubicacion'] = ubicacion
    
    # - Link de la empresa
    link_empresa = oferta.find('a', {'class':'hidden-nested-link'}).attrs['href']
    dicc_oferta['link_empresa'] = link_empresa

    # - Link de la oferta
    link_oferta = oferta.find('a', {'class':'base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]'}).attrs['href']
    dicc_oferta['link_oferta'] = link_oferta
    # - Fecha de publicacion
    fecha_publicacion = oferta.find('time', {'job-search-card__listdate--new'}).text.strip()
    dicc_oferta['fecha_publicacion'] = fecha_publicacion

    #Appendeamos a nuestra lista de diccionarios de respuesta
    lista_ofertas.append(dicc_oferta)    
    

In [18]:
dicc_oferta

{'titulo_oferta': 'Data Entry Clerk ( Remote 100%)',
 'empresa': 'Opinion Focus Panel',
 'ubicacion': 'New York, United States',
 'link_empresa': 'https://www.linkedin.com/company/opinionfocuspanelllc?trk=public_jobs_jserp-result_job-search-card-subtitle',
 'link_oferta': 'https://www.linkedin.com/jobs/view/data-entry-clerk-remote-100%25-at-opinion-focus-panel-4026274571?position=3&pageNum=0&refId=JhuVAKYv03npP4%2BbCs0mzw%3D%3D&trackingId=PyxWpYqv5%2BBdm0yjjeSo7A%3D%3D&trk=public_jobs_jserp-result_search-card',
 'fecha_publicacion': '6 hours ago'}

In [44]:
lista_ofertas

[{'titulo_oferta': 'Entry Level Data Analyst',
  'empresa': 'Escola de Economia e Gestão - Universidade do Minho',
  'ubicacion': 'United States',
  'link_empresa': 'https://pt.linkedin.com/school/eegum/?trk=public_jobs_jserp-result_job-search-card-subtitle'},
 {'titulo_oferta': 'Data Analyst',
  'empresa': 'Opinion Focus Panel',
  'ubicacion': 'Virginia, United States',
  'link_empresa': 'https://www.linkedin.com/company/opinionfocuspanelllc?trk=public_jobs_jserp-result_job-search-card-subtitle'},
 {'titulo_oferta': 'Data Entry Clerk ( Remote 100%)',
  'empresa': 'Opinion Focus Panel',
  'ubicacion': 'New York, United States',
  'link_empresa': 'https://www.linkedin.com/company/opinionfocuspanelllc?trk=public_jobs_jserp-result_job-search-card-subtitle'}]

In [16]:
oferta.find('a', {'class':'base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]'}).attrs['href']

'https://www.linkedin.com/jobs/view/data-entry-clerk-remote-100%25-at-opinion-focus-panel-4026274571?position=3&pageNum=0&refId=JhuVAKYv03npP4%2BbCs0mzw%3D%3D&trackingId=PyxWpYqv5%2BBdm0yjjeSo7A%3D%3D&trk=public_jobs_jserp-result_search-card'

In [15]:
ofertas[1].find('time', {'job-search-card__listdate--new'}).text.strip()


'6 hours ago'

In [ ]:
#Os recomiendo ir echando una ojeada al html y la sopa poco a poco para ver como esta estructurado
#Lo primero que tendremos que buscar será el elemento que contiene todas las ofertas....🤔

#PISTA -- base-search-card__info
#De aqui en adelante vosotros solos, tened presente que querremos guardar toda esa info en un dataframe

### REDDIT - https://www.reddit.com/r/Python/

In [34]:
url = 'https://www.reddit.com/r/Python/'
html = req.get(url).content
soup = bs(html, 'html.parser')

In [35]:
print(soup.prettify())

<!DOCTYPE html>
<html class="theme-beta" lang="en-US">
 <head prefix="og: https://ogp.me/ns#">
  <title>
   Reddit - Dive into anything
  </title>
  <meta content="width=device-width, initial-scale=1, viewport-fit=cover" name="viewport"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <meta content="black" name="apple-mobile-web-app-status-bar-style"/>
  <meta content="#000000" name="msapplication-navbutton-color"/>
  <meta content="#000000" name="theme-color"/>
  <meta content="noarchive" name="bingbot">
   <link crossorigin="anonymous" href="https://www.redditstatic.com/" rel="preconnect"/>
   <link href="https://www.redditstatic.com/" rel="dns-prefetch"/>
   <script nonce="4hL0KcBvUreBHnvedpoGIg==" type="module">
    !function(e,n,o,t,i,r,d){var a=!1,c=!1,s=[],f=function(e){("e"in e||"p"in e||e.f&&e.f.indexOf("capture")>-1||e.f&&e.f.indexOf("withScope")>-1||e.f&&e.f.indexOf("showReportDialog")>-1)&&u(s),f.data.push(e)};function u(n){function o(){import("https://www.re

In [40]:
#Vamos a sacar info de los artículos de la página de Reddit

In [37]:
#De cada artículo vamos a sacar su autor

3

In [41]:
#Vamos a sacar el tipo de artículo

In [ ]:
#Tipo articulo

Cuando tengo una etiqueta sea la que sea dentro tengo su clase, id, y más atributos. href, title, target...

El contenido de los atributos también es accesible.

In [ ]:
#Vamos a coger la url de cada uno de los artículos
